In [4]:
from datasets import load_dataset

silver_data = load_dataset('json', data_files='data/silver_data/*')

Using custom data configuration default-43742e70be1ca76a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/14cfl/.cache/huggingface/datasets/json/default-43742e70be1ca76a/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
[(print(x) if x.startswith('Yes') else None) for x in silver_data['train']['output']]

Yes, this post could potentially violate TOS_MET, TOS_RED, TOS_TK, TOS_TWT, and TOS_YT. These policies prohibit content that attacks, threatens, or dehumanizes an individual or a group based on protected attributes, which include sex and gender. The statement "fuck men" can be seen as an attack or expression of contempt towards men, which is a group defined by their sex/gender. However, it's important to note that the context
Yes, this post potentially violates the policies of CC_319, HR_AB, HR_BC, HR_NW, HR_SA, TOS_MET, TOS_RED, TOS_TK, TOS_TWT, and TOS_YT. These policies protect against discrimination or incitement of hatred based on religion. The post targets Mormons, a religious group, suggesting they should be evicted, which can be seen as promoting discrimination or hatred.
Yes, this post could potentially violate TOS_MET, TOS_RED, TOS_TK, TOS_TWT, and TOS_YT. The statement seems to imply a threat or violence against an individual based on an unspecified characteristic (possibly 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [8]:
import pandas as pd

legal_defs = pd.read_csv('legal definitions.csv')
legal_labels = legal_defs['name'].to_list()
defs = pd.read_csv('data/definitions.csv')

In [10]:
prompt = ['{target} states "{target_def}" Protected groups include {targets}'.format(
    target=legal_defs.iloc[i]['promptName'],
    target_def=legal_defs.iloc[i]['definition'],
    targets=legal_defs.iloc[i]['protected_groups']
) for i in range(len(legal_defs))]

legal_defs['prompt'] = prompt

In [19]:
from nltk.tokenize import word_tokenize

new_data = {'instruction': [], 'input': [], 'output': [] 'label': [], 'finegrained': []}

for i in range(len(legal_defs)):
    for d in silver_data['train']:
        new_data['instruction'].append(legal_defs.iloc[i]['prompt'])
        
        new_data['input'].append(d['input'].split('Q: ')[0] + ' Q: Is this post a violation of {target}? Please answer Yes, No, or Unclear. A: '.format(target=legal_defs.iloc[i]['promptName']))
        temp = []
        bag_of_words = set(word_tokenize(d['output']))
        if d['output'].startswith('Yes'):
            if legal_defs.iloc[i]['promptName'] in bag_of_words:
                temp.append(1)
                if '1.' not in bag_of_words:
                    new_data['output'].append('Yes. '+ d['output'].split('.', 1)[-1])
                else:
                    new_data['output'].append('Yes. '+ ''.join([x for x in d['output'].split() if legal_defs.iloc[i]['promptName'] in x]))
            else:
                new_data['output'].append('No, but the statement could be considered hateful under another policy.')
                temp.append(0)
                
            
        elif d['output'].startswith('Unclear'):
            new_data['label'].append(1)
            new_data['output'].append(d['output'])
    

In [20]:
from datasets import Dataset

Dataset.from_dict(new_data).to_json('data/silver_data_singletarget.json')

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

36991204

In [21]:
multi_target = {'instruction': ['\n'.join(prompt)]*len(silver_data['train']['instruction']), 
                'input': silver_data['train']['input'], 
                'output': silver_data['train']['input']}



In [22]:
Dataset.from_dict(multi_target).to_json('data/silver_data_multitarget.json')

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

25447872

In [26]:
for_df = { 'text': [], 'label': [], 'finegrained': []}

for d in silver_data['train']:
    for_df['text'].append(d['input'].split('Q: ')[0].split('Post:')[-1] )
    
    bag_of_words = set(word_tokenize(d['output']))
    if d['output'].startswith('Yes'):
        temp = []
        for i in range(len(legal_defs)):
            if legal_defs.iloc[i]['promptName'] in bag_of_words:
                temp.append(1)
            else:
                temp.append(0)
        for_df['finegrained'].append(temp)
        for_df['label'].append(2)
            
        
    elif d['output'].startswith('Unclear'):
        for_df['label'].append(1)
        for_df['finegrained'].append(None)
    else:
        for_df['label'].append(0)
        for_df['finegrained'].append(None)

        

In [27]:
pd.DataFrame.from_dict(for_df).to_csv('silverdata.csv')